## Market Impact and Trade Sizing
J. White \
8 March 2020

We want to explore the effects of trading impact and discrete 'trap-door' scenarios in a classic Merton/Samuelson sizing framework:

### Assumptions
* We can only invest in one risky asset S and a risk-free asset R with 0 expected return
* $\frac{\Delta S_T}{S_0}$ can take values $x_i$ with probabilities $p_i$
* Investor has CRRA utility: $$u(w) = \frac{w^{1-\gamma}}{1-\gamma}$$
* Trading impact is treated as a tax, which is a function of trade size and execution duration: $$\tau = f(\kappa, \Delta t)$$
* $\tau$ is symmetric between entering and exiting
* No rebalancing between times 0 and T

### General Framework
For a wealth fraction $\kappa$ invested in $S$, we have expected utility:
$$E[u] = \frac{1}{1-\gamma}\sum_i p_i (1+\kappa x_i - 2\tau(\kappa))^{1-\gamma}$$

As usual we want to optimize wrt $\kappa$ so:
$$\frac{\partial E[u]}{\partial \kappa} = \sum_i \frac{p_i (x_i - 2\frac{\partial\tau}{\partial\kappa})}{(1+\kappa x_i - 2\tau(\kappa))^{\gamma}}  = 0 $$

This can be solved analytically for certain distributions and $\tau$ specs, but in general we'll just solve it numerically.

### Special Case: Normal returns
We know if $p(x) \approx N(\mu, \sigma)$ and the risk-free asset has rate $r$, then the optimal wealth fraction is:
$$\kappa = \frac{\mu - r}{\gamma\sigma^2}$$

If $\tau = a\kappa$ it's equivalent to just adjusting $\mu$, so the ratio of optimal size with/without trade impact is simply
$1 - \frac{2a}{\mu - r}$

### Extensions
* What is the best form for $\tau$, and is it symmetric?
* Here we've suppressed the presumed dependence of $\tau$ on $\Delta t$, but it would be interesting to explore optimal trade speed under realistic assumptions about alpha decay etc.
* Multiple assets with distinct trade cost functions
* Rebalancing effects
* Dynamic scaling rule incorporating actual trade costs

### Interactive Code

**Note: I've used a very rudminentary linear function for trade cost - to use any other 1-parameter trade cost function just replace the definition of *tau* here:**

In [152]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, BoundedFloatText, interactive_output, HBox

# Definitions
util = lambda x: (pow(x, 1 - gamma) - 1) / (1 - gamma)

def tau(k, a):
    return a * k

def expected_util(p, x, k, a, gamma):
    return sum(p * (pow((1 + k * x - 2 * tau(k, a)), 1 - gamma) - 1)) / (1 - gamma)

# Output
def plot_utils(p0, p1, p2 ,p3, p4, x0, x1, x2, x3, x4, tau_strength, gamma):
    # Compute expected utility with and without trade-impact 
    p0 = float(p0)
    p = np.asarray([p0, p1, p2, p3, p4]) / (p0 + p1 + p2 + p3 + p4)
    x = np.asarray([x0, x1, x2, x3, x4])
    dk = 0.02
    N = 100
    eU_base = np.zeros(N)
    eU_impact = np.zeros(N)
    x_axis = np.zeros(N)
    for i,k in enumerate(range(0, N)):
        x_axis[i] = dk * k
        eU_base[i] = expected_util(p, x, dk * k, 0, gamma)
        eU_impact[i] = expected_util(p, x, dk * k, tau_strength, gamma)
    
    # The main utility plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))    
    ax1.plot(x_axis, eU_base, 'b', label='E[u] No Trade Impact')
    ax1.plot(x_axis, eU_impact, 'g', label='E[u] With Trade Impact')
    ax1.plot(x_axis, np.dot(p, x) * x_axis, 'r', label='Expected Return')
    ax1.plot(x_axis, np.zeros(N), 'k')
    ax1.set_xlabel('Wealth % (Trade Size)')
    ax1.set_ylabel('Risk-adjusted Return')
    ax1.legend()
    ax1.set_title('Risk-adjusted Return vs. Trade Size')
    ax1.spines['bottom'].set_position(('data', 0))
    
    # Plot gradients also
    ax2.plot(x_axis, np.gradient(eU_base), 'b', label='No Trade Impact')
    ax2.plot(x_axis, np.gradient(eU_impact), 'g', label='With Trade Impact')
    ax2.plot(x_axis, np.zeros(N), 'k')
    ax2.set_xlabel('Wealth % (Trade Size)')
    ax2.set_ylabel('Risk-adjusted Return / Size %')
    ax2.legend()
    ax2.set_title('Gradient wrt trade size')
    ax2.spines['bottom'].set_position(('data', 0))
    fig.tight_layout()
    
    # Some extra info for context
    print('Expected Return (pre-Impact) = {}'.format(np.dot(p,x)))
    print('Trade Impact at 100% size = {}'.format(2 * tau(1, tau_strength)))

In [153]:
# Set up the interactive plots
s1 = BoundedFloatText(min=1.01, max=10, step=0.25, value=2, description='gamma')
s2 = BoundedFloatText(min=0, max=1, step=0.001, value=0.02, description='tau_strength')
s3 = BoundedFloatText(min=0, max=1, step=0.05, value=0.05, description='p0')
s4 = BoundedFloatText(min=0, max=1, step=0.05, value=0.20, description='p1')
s5 = BoundedFloatText(min=0, max=1, step=0.05, value=0.5, description='p2')
s6 = BoundedFloatText(min=0, max=1, step=0.05, value=0.20, description='p3')
s7 = BoundedFloatText(min=0, max=1, step=0.05, value=0.05, description='p4')
s8 = BoundedFloatText(min=-1, max=1, step=0.05, value=-0.3, description='x0')
s9 = BoundedFloatText(min=-1, max=1, step=0.05, value=-0.1, description='x1')
s10 = BoundedFloatText(min=-1, max=1, step=0.05, value=0.1, description='x2')
s11 = BoundedFloatText(min=-1, max=1, step=0.05, value=0.3, description='x3')
s12 = BoundedFloatText(min=-1, max=1, step=0.05, value=0.5, description='x4')
plots = interactive_output(plot_utils, {'gamma':s1, 'tau_strength':s2, 'p0':s3, 'p1':s4, 'p2':s5, 'p3':s6, 'p4':s7, 
                                      'x0':s8, 'x1':s9, 'x2':s10, 'x3':s11, 'x4':s12})

### Interactive Utlity Graphs
* $p_i$ inputs are unconstrained then $\bar{p}$ is normalized behind the scenes, so $p$ inputs can be interpreted as relative probability weights
* It can be tricky relating trade size in this thought experiement with only one risky asset/trade to real-world cases with many correlated/uncorrelated trades.  One way is to select $\gamma$ (risk-aversion) so that a well-known 'reference' trade has optimal size 1, then sizes for all others trades can be thought of as *relative* sizes vs. the reference trade

In [154]:
display(HBox([s3, s4, s5, s6, s7]))
display(HBox([s8, s9, s10, s11, s12]))
display(HBox([s1,s2]))
display(plots)

Output(outputs=({'output_type': 'stream', 'text': 'Expected Return (pre-Impact) = 0.1\nTrade Impact at 100% si…